In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/scrabble-player-rating/sample_submission.csv
/kaggle/input/scrabble-player-rating/games.csv
/kaggle/input/scrabble-player-rating/turns.csv
/kaggle/input/scrabble-player-rating/train.csv
/kaggle/input/scrabble-player-rating/test.csv


In [2]:
# reading all necessary files into the notebook
games = pd.read_csv('/kaggle/input/scrabble-player-rating/games.csv')
turns = pd.read_csv('/kaggle/input/scrabble-player-rating/turns.csv')
train = pd.read_csv('/kaggle/input/scrabble-player-rating/train.csv')
test = pd.read_csv('/kaggle/input/scrabble-player-rating/test.csv')

In [3]:
print('Shape of game dataframe is:', games.shape)
print('Shape of turns dataframe is:', turns.shape)
print('Shape of train dataframe is:', train.shape)
print('Shape of test dataframe is:', test.shape)

Shape of game dataframe is: (72773, 12)
Shape of turns dataframe is: (2005498, 9)
Shape of train dataframe is: (100820, 4)
Shape of test dataframe is: (44726, 4)


In [4]:
games.head()

,game_id,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
0,1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274
1,2,Super,regular,STANDARD,1,2022-08-10 19:19:59,CSW21,3600,0,RATED,1,364.214418
2,3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262
3,4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141
4,5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722


In [5]:
turns.head()

,game_id,turn_number,nickname,rack,location,move,points,score,turn_type
0,1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
2,1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
3,1,4,stevy,EMORSUX,5K,UM,16,34,Play
4,1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play


In [6]:
train.isna().sum()

game_id     0
nickname    0
score       0
rating      0
dtype: int64

In [7]:
train.head()

,game_id,nickname,score,rating
0,1,BetterBot,335,1637
1,1,stevy,429,1500
2,3,davidavid,440,1811
3,3,BetterBot,318,2071
4,4,Inandoutworker,119,1473


In [8]:
bots = ['BetterBot', 'STEEBot', 'HastyBot']

In [9]:
users_df = train[~train['nickname'].isin(bots)]
users_df = users_df.rename(columns={'nickname': 'user', 'score': 'user_score', 'rating': 'user_rating'})
users_df.head()

,game_id,user,user_score,user_rating
1,1,stevy,429,1500
2,3,davidavid,440,1811
4,4,Inandoutworker,119,1473
6,5,stevy,325,1500
8,6,HivinD,378,2029


In [10]:
bots_df = train[train['nickname'].isin(bots)]
bots_df = bots_df.rename(columns={'nickname': 'bots', 'score': 'bots_score', 'rating': 'bots_rating'})
bots_df.head()

,game_id,bots,bots_score,bots_rating
0,1,BetterBot,335,1637
3,3,BetterBot,318,2071
5,4,BetterBot,478,1936
7,5,STEEBot,427,1844
9,6,STEEBot,427,2143


In [11]:
main_df = pd.merge(users_df, bots_df, on='game_id')
main_df

,game_id,user,user_score,user_rating,bots,bots_score,bots_rating
0,1,stevy,429,1500,BetterBot,335,1637
1,3,davidavid,440,1811,BetterBot,318,2071
2,4,Inandoutworker,119,1473,BetterBot,478,1936
3,5,stevy,325,1500,STEEBot,427,1844
4,6,HivinD,378,2029,STEEBot,427,2143
...,...,...,...,...,...,...,...
50405,72767,HAYDEN,340,1395,BetterBot,414,1952
50406,72770,samsiah06,97,1332,HastyBot,590,2237
50407,72771,BB-8,390,1500,HastyBot,393,1614
50408,72772,Gtowngrad,388,1364,BetterBot,442,1674


In [12]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50410 entries, 0 to 50409
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   game_id      50410 non-null  int64 
 1   user         50410 non-null  object
 2   user_score   50410 non-null  int64 
 3   user_rating  50410 non-null  int64 
 4   bots         50410 non-null  object
 5   bots_score   50410 non-null  int64 
 6   bots_rating  50410 non-null  int64 
dtypes: int64(5), object(2)
memory usage: 3.1+ MB


In [13]:
main_df.describe(include='all')

,game_id,user,user_score,user_rating,bots,bots_score,bots_rating
count,50410.000000,50410,50410.000000,50410.000000,50410,50410.000000,50410.000000
unique,NaN,1028,NaN,NaN,3,NaN,NaN
top,NaN,BB-8,NaN,NaN,HastyBot,NaN,NaN
freq,NaN,6576,NaN,NaN,17346,NaN,NaN
mean,36367.328605,NaN,376.060722,1790.322674,NaN,421.997322,1960.863202
std,21020.392953,NaN,72.739107,229.662441,NaN,70.423928,200.419697
min,1.000000,NaN,-64.000000,1033.000000,NaN,162.000000,1349.000000
25%,18139.750000,NaN,331.000000,1582.000000,NaN,373.000000,1827.000000
50%,36339.500000,NaN,378.000000,1818.000000,NaN,417.000000,1991.000000
75%,54624.750000,NaN,423.000000,1982.000000,NaN,465.000000,2124.000000


In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
lab_encode = LabelEncoder()
main_df['bots'] = lab_encode.fit_transform(main_df['bots'])

In [16]:
main_df.head()

,game_id,user,user_score,user_rating,bots,bots_score,bots_rating
0,1,stevy,429,1500,0,335,1637
1,3,davidavid,440,1811,0,318,2071
2,4,Inandoutworker,119,1473,0,478,1936
3,5,stevy,325,1500,2,427,1844
4,6,HivinD,378,2029,2,427,2143
